In [2]:
import numpy as np 
import pandas as pd 
from datetime import datetime, date, timedelta
from tqdm import tqdm_notebook as tqdm
import os
import pymssql as mssql
import calendar
import json
from multiprocessing import Pool    
from functools import partial 

import Online_label as func

In [39]:
d = func.stock_query(date(2017,8,1), date(2019,9,20))
d['ts'] = pd.to_datetime(d['ts'])

In [22]:
origin_path = 'D:\\庫存健診開發\\data\\processed\\TWSE_feature_all_5day.csv'
df = pd.read_csv(origin_path, converters={'StockNo': str}, usecols=['ts', 'StockNo', 'Y', 'VWAP_day5'])
df['ts'] = pd.to_datetime(df['ts'])

In [40]:
d = d.rename(columns={'Y': 'SQL_Y'})

In [41]:
data = pd.merge(df, d, on=['ts', 'StockNo'], how='inner')

In [42]:
def check(row):
    if row['Y'] == row['SQL_Y']:
        return True
    else:
        return False

data['check'] = data.apply(check, axis=1)
        

In [56]:
d1 = data[(data.ts.dt.date >= date(2017,9,1)) & (data.ts.dt.date < date(2019,9,8)) & (~data['StockNo'].isin(['1449', '6133']))]
d1[~d1.check].iloc[45:100]

,ts,StockNo,VWAP_day5,Y,PAST_VWAP_5D,FUT_VWAP_5D,SQL_Y,check
30879,2018-10-19,1423,10.298788,1.0,10.298788,0.000000,0,False
39914,2017-09-25,1451,23.747253,1.0,23.747253,23.367918,0,False
39915,2017-09-26,1451,23.544761,1.0,23.544761,23.333924,0,False
39916,2017-09-27,1451,23.465455,1.0,23.465455,23.272109,0,False
39917,2017-09-28,1451,23.364919,1.0,23.364919,0.000000,0,False
39918,2017-09-29,1451,23.361793,1.0,23.361793,0.000000,0,False
47791,2018-04-30,1470,18.166667,1.0,18.166667,0.000000,0,False
47796,2018-05-08,1470,18.333333,0.0,0.000000,18.000000,1,False
47805,2018-05-21,1470,20.333333,1.0,20.333333,0.000000,0,False
47806,2018-05-22,1470,21.500000,1.0,21.500000,0.000000,0,False


In [50]:
d1[d1.StockNo == '1451'].iloc[:50]

,ts,StockNo,VWAP_day5,Y,PAST_VWAP_5D,FUT_VWAP_5D,SQL_Y,check
39898,2017-09-01,1451,22.180052,0.0,22.180052,22.156375,0,True
39899,2017-09-04,1451,22.137602,1.0,22.137602,22.229808,1,True
39900,2017-09-05,1451,22.164609,1.0,22.164609,22.348575,1,True
39901,2017-09-06,1451,22.138372,1.0,22.138372,22.489736,1,True
39902,2017-09-07,1451,22.153605,1.0,22.153605,22.645072,1,True
39903,2017-09-08,1451,22.156375,1.0,22.156375,22.881272,1,True
39904,2017-09-11,1451,22.229808,1.0,22.229808,23.602937,1,True
39905,2017-09-12,1451,22.348575,1.0,22.348575,23.845973,1,True
39906,2017-09-13,1451,22.489736,1.0,22.489736,23.928651,1,True
39907,2017-09-14,1451,22.645072,1.0,22.645072,23.969690,1,True
